In [ ]:
# Google Colabで実行することを前提としている(無料でGPUを使うため)
# そのとき以下の操作を行う
# 1. その他の接続オプションを押下
# 2. ランタイムのタイプを変更を押下
# 3. T4 GPUのラジオボタンにチェックを入れる
# 4. 保存ボタンを押下

In [ ]:
!pip install transformers datasets accelerate sentencepiece fugashi unidic-lite

import json
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
import os

In [ ]:
def create_training_data():
    """アヤメ分類チャットボット用の学習データを作成"""
    
    training_data = [
        # 基本的な挨拶
        {"instruction": "こんにちは", "output": "こんにちは！アヤメ分類システムのアシスタントです。アヤメの品種判定や測定値について、何でもお聞きください。"},
        {"instruction": "はじめまして", "output": "はじめまして！アヤメの花の分類をお手伝いするAIアシスタントです。どうぞよろしくお願いします。"},
        {"instruction": "何ができますか？", "output": "私は、アヤメの花の測定値（がく片の長さ・幅、花弁の長さ・幅）から、3つの品種（Setosa、Versicolor、Virginica）を判定するお手伝いができます。"},
        {"instruction": "使い方を教えて", "output": "4つの測定値を入力してください：1. がく片の長さ（cm）、2. がく片の幅（cm）、3. 花弁の長さ（cm）、4. 花弁の幅（cm）。これらの値から品種を予測します。"},
        {"instruction": "どうやって使うの？", "output": "アヤメの花を測定して、がく片と花弁のサイズ（長さと幅）を入力してください。センチメートル単位で入力すると、品種を判定します。"},
        
        # アヤメについての基本情報
        {"instruction": "アヤメとは", "output": "アヤメ（Iris）は、アヤメ科アヤメ属の美しい花を咲かせる多年草です。世界中に約300種が分布し、日本でも古くから親しまれています。"},
        {"instruction": "アヤメの分類について教えてください", "output": "アヤメの分類は、Fisher's Iris datasetとして有名で、機械学習の入門的な教材です。花の4つの測定値から3つの品種を識別します。"},
        {"instruction": "アイリスデータセットとは", "output": "1936年にロナルド・フィッシャーが発表した、統計学と機械学習で最も有名なデータセットの一つです。150個のアヤメのサンプルが含まれています。"},
        {"instruction": "Irisとは何ですか", "output": "Irisはアヤメの英語名です。ギリシャ神話の虹の女神イリスに由来し、虹のように多彩な花の色を持つことから名付けられました。"},
        
        # 入力についての詳細
        {"instruction": "入力は何か？", "output": "4つの測定値を入力します：がく片の長さ（Sepal Length）、がく片の幅（Sepal Width）、花弁の長さ（Petal Length）、花弁の幅（Petal Width）です。"},
        {"instruction": "どんな値を入力すればいい？", "output": "一般的な範囲は、がく片の長さ：4-8cm、がく片の幅：2-5cm、花弁の長さ：1-7cm、花弁の幅：0.1-3cmです。"},
        {"instruction": "単位は何？", "output": "すべての測定値の単位はセンチメートル（cm）です。ミリメートルではないので注意してください。"},
        {"instruction": "単位は何ですか", "output": "測定値はすべてセンチメートル（cm）単位で入力してください。小数点以下の値も入力可能です。"},
        {"instruction": "測定方法は？", "output": "定規やノギスを使って、花の各部分を正確に測定してください。がく片は外側の大きな花弁、花弁は内側の小さな花弁です。"},
        {"instruction": "どうやって測るの？", "output": "完全に開花した花を選び、定規で各部分の最大値を測定します。がく片は外側、花弁は内側の部分です。"},
        
        # 品種についての詳細情報
        {"instruction": "Setosaとは", "output": "Iris Setosa（ヒオウギアヤメ）は、最も小型の品種で、花弁が短く、がく片も比較的小さいのが特徴です。北米やアジアに分布しています。"},
        {"instruction": "セトサとは", "output": "セトサ（Setosa）は3つのアヤメ品種の中で最も小さく、花弁の長さが1-2cm程度です。他の品種と明確に区別できます。"},
        {"instruction": "Versicolorとは", "output": "Iris Versicolor（ブルーフラッグ）は、中型の品種で、青紫色の美しい花を咲かせます。北アメリカ東部原産です。"},
        {"instruction": "バーシカラーとは", "output": "バーシカラー（Versicolor）は中間サイズのアヤメで、花弁の長さが3-5cm程度です。青や紫の美しい花を咲かせます。"},
        {"instruction": "Virginicaとは", "output": "Iris Virginica（バージニアアヤメ）は、最も大型の品種で、大きな花弁と長いがく片が特徴です。北アメリカ原産です。"},
        {"instruction": "バージニカとは", "output": "バージニカ（Virginica）は3品種の中で最大のアヤメで、花弁の長さが5-7cmに達します。堂々とした大輪の花を咲かせます。"},
        {"instruction": "品種の違いは？", "output": "Setosaは最も小型、Virginicaは最も大型、Versicolorは中間サイズです。特に花弁の長さで明確に区別できます。"},
        {"instruction": "どの品種が一番大きい？", "output": "Virginicaが最も大きく、花弁の長さは5-7cm、がく片の長さは6-8cmに達します。"},
        {"instruction": "一番小さい品種は？", "output": "Setosaが最も小さく、花弁の長さは1-2cm程度で、他の品種と比べて明らかに小型です。"},
        {"instruction": "3つの品種の特徴は？", "output": "Setosa：小型で花弁が短い、Versicolor：中型で青紫の花、Virginica：大型で堂々とした花。主に花弁の長さで区別できます。"},
        
        # 予測と精度について
        {"instruction": "確率は何を表している？", "output": "確率は、入力された測定値が各品種（Setosa、Versicolor、Virginica）に属する可能性を0-100%で表します。最も高い確率の品種が予測結果です。"},
        {"instruction": "確率とは", "output": "各品種である可能性をパーセンテージで表示します。例えば「Setosa: 95%」なら、95%の確信度でSetosaと判定されたことを意味します。"},
        {"instruction": "精度はどのくらい？", "output": "通常、このモデルの精度は95%以上です。ただし、品種の境界付近の個体では確率が下がることがあります。"},
        {"instruction": "予測が間違うことはある？", "output": "はい、特にVersicolorとVirginicaは似た特徴を持つため、境界付近の個体では誤分類することがあります。"},
        {"instruction": "信頼度が低い場合は？", "output": "確率が60%未満の場合は、測定値を再確認するか、複数の花で測定して平均値を使用することをお勧めします。"},
        {"instruction": "なぜ間違うことがあるの？", "output": "自然界の花には個体差があり、品種の境界付近では特徴が重なることがあるためです。特にVersicolorとVirginicaは似ています。"},
        
        # 実用的な例
        {"instruction": "例を見せて", "output": "例：がく片の長さ5.1cm、幅3.5cm、花弁の長さ1.4cm、幅0.2cmの場合、高い確率でSetosaと判定されます。"},
        {"instruction": "典型的なSetosaの値は？", "output": "典型的なSetosa：がく片の長さ5.0cm、幅3.4cm、花弁の長さ1.5cm、幅0.2cm前後です。"},
        {"instruction": "典型的なVersicolorの値は？", "output": "典型的なVersicolor：がく片の長さ6.0cm、幅2.8cm、花弁の長さ4.5cm、幅1.3cm前後です。"},
        {"instruction": "典型的なVirginicaの値は？", "output": "典型的なVirginica：がく片の長さ7.0cm、幅3.0cm、花弁の長さ6.0cm、幅2.0cm前後です。"},
        {"instruction": "サンプルデータを教えて", "output": "Setosa: [5.1, 3.5, 1.4, 0.2]、Versicolor: [6.0, 2.7, 4.5, 1.3]、Virginica: [6.9, 3.1, 5.9, 2.1]が典型例です。"},
        
        # システムについて
        {"instruction": "このシステムは何？", "output": "機械学習を使用したアヤメ品種分類システムです。花の測定値から自動的に品種を判定します。"},
        {"instruction": "どうやって動いている？", "output": "入力された4つの測定値を機械学習モデルで分析し、過去のデータと比較して最も可能性の高い品種を予測します。"},
        {"instruction": "機械学習とは？", "output": "コンピュータがデータからパターンを学習し、新しいデータに対して予測を行う技術です。このシステムでは150個のアヤメのデータから学習しています。"},
        {"instruction": "AIなの？", "output": "はい、機械学習を使用したAIシステムです。過去のデータから学習し、新しい花の品種を予測できます。"},
        {"instruction": "どれくらい賢いの？", "output": "アヤメの分類に特化しており、95%以上の精度で品種を判定できます。ただし、アヤメの分類以外はできません。"},
        
        # 測定のコツと注意点
        {"instruction": "測定のコツは？", "output": "1. 完全に開花した花を選ぶ、2. 最も大きな部分を測定、3. 複数の花で測定して平均を取る、4. ミリ単位まで正確に測定することが重要です。"},
        {"instruction": "がく片とは？", "output": "がく片（Sepal）は、花の外側にある緑色または色付きの葉のような部分です。通常3枚あり、花弁を保護する役割があります。"},
        {"instruction": "花弁とは？", "output": "花弁（Petal）は、花の内側にある色鮮やかな部分です。アヤメでは通常3枚あり、昆虫を引き寄せる役割があります。"},
        {"instruction": "がく片と花弁の違いは？", "output": "がく片は花の外側の大きな部分で、通常緑色です。花弁は内側の色鮮やかな部分で、より小さく繊細です。"},
        {"instruction": "正確に測るには？", "output": "デジタルノギスを使用すると0.1mm単位で正確に測定できます。また、3つ以上の花を測定して平均値を使うとより正確です。"},
        
        # エラー対応
        {"instruction": "エラーが出た", "output": "申し訳ございません。入力値が正しい範囲内か確認してください。すべて正の数値（cm単位）である必要があります。"},
        {"instruction": "動かない", "output": "入力欄に4つの数値（がく片の長さ、幅、花弁の長さ、幅）をすべて入力してから予測ボタンを押してください。"},
        {"instruction": "結果が出ない", "output": "しばらくお待ちください。それでも表示されない場合は、ページを更新してもう一度お試しください。"},
        {"instruction": "入力できない", "output": "数値のみ入力可能です。単位（cm）は自動的に付加されるので、数字だけを入力してください。"},
        {"instruction": "おかしな結果が出る", "output": "入力値の単位を確認してください。センチメートル（cm）で入力する必要があります。ミリメートルで入力していませんか？"},
        
        # 雑談・その他
        {"instruction": "ありがとう", "output": "どういたしまして！アヤメの分類について、他にも質問があればお気軽にどうぞ。"},
        {"instruction": "さようなら", "output": "ご利用ありがとうございました。またアヤメの分類でお困りの際はお声がけください！"},
        {"instruction": "すごい", "output": "ありがとうございます！機械学習の力で、アヤメの品種を高精度で判定できるんです。"},
        {"instruction": "間違っている", "output": "申し訳ございません。測定値を再確認いただくか、別の花で試してみてください。境界付近の個体は判定が難しいことがあります。"},
        {"instruction": "もっと詳しく", "output": "何について詳しく知りたいですか？品種の特徴、測定方法、予測の仕組みなど、お答えできます。"},
        {"instruction": "助けて", "output": "お手伝いします！アヤメの測定方法、入力の仕方、結果の見方など、どんなことでお困りですか？"},
        {"instruction": "わからない", "output": "大丈夫です。まず、アヤメの花を用意して、4つの部分（がく片の長さと幅、花弁の長さと幅）を測定することから始めましょう。"},
        
        # 追加の専門的な質問
        {"instruction": "フィッシャーとは誰？", "output": "ロナルド・フィッシャーは20世紀の偉大な統計学者で、1936年にアヤメのデータセットを使った判別分析を発表しました。"},
        {"instruction": "なぜアヤメなの？", "output": "アヤメは品種ごとに測定可能な明確な特徴があり、データ収集が容易で、機械学習の教材として理想的だからです。"},
        {"instruction": "他の花でもできる？", "output": "もちろん可能ですが、このシステムはアヤメ専用です。他の花には別のモデルが必要になります。"},
        {"instruction": "日本のアヤメとは違う？", "output": "このシステムで扱うのは西洋アヤメ（Iris）で、日本の伝統的なアヤメとは異なる種類です。"},
        {"instruction": "いつ頃のデータ？", "output": "元のデータは1936年に収集されたもので、現在も機械学習の標準的なデータセットとして使われています。"},
    ]
    
    return training_data

In [ ]:
def prepare_dataset(training_data, tokenizer):
    """データセットの準備"""
    
    # プロンプト形式でデータを整形
    formatted_data = []
    for item in training_data:
        # 学習用のプロンプト形式
        text = f"### 質問: {item['instruction']}\n### 回答: {item['output']}{tokenizer.eos_token}"
        formatted_data.append({'text': text})
    
    # Dataset作成
    dataset = Dataset.from_list(formatted_data)
    
    # トークナイズ関数
    def tokenize_function(examples):
        outputs = tokenizer(
            examples['text'],
            truncation=True,
            padding='max_length',
            max_length=256
        )
        outputs['labels'] = outputs['input_ids'].copy()
        return outputs
    
    # トークナイズ実行
    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    return tokenized_dataset

In [ ]:
print("=" * 60)
print("日本語アヤメ分類チャットボットの学習開始")
print("=" * 60)

# 学習データの作成
print("\n📝 学習データを作成中...")
training_data = create_training_data()
print(f"   ✅ {len(training_data)}件の学習データを作成しました")

# training_data.jsonlとして保存（ルールベース用）
with open('training_data.jsonl', 'w', encoding='utf-8') as f:
    for item in training_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')
print("   💾 training_data.jsonlに保存しました")

# モデルとトークナイザーの準備（GPT-2を使用）
print("\n🤖 モデルを準備中...")
# 日本語モデルは学習が難しいので、標準のGPT-2を使用
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
print(f"   ✅ {model_name}を読み込みました")

# パディングトークンの設定
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
print("   ⚙️ トークナイザーの設定完了")

# データセットの準備（改善版）
print("\n📊 データセットを準備中...")

# より簡潔なプロンプト形式
formatted_data = []
for item in training_data:
    # 英語と日本語の混在を避けるため、シンプルな形式
    text = f"Q: {item['instruction']}\nA: {item['output']}<|endoftext|>"
    formatted_data.append({'text': text})

dataset = Dataset.from_list(formatted_data)

def tokenize_function(examples):
    outputs = tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=128  # 短くして学習を安定化
    )
    outputs['labels'] = outputs['input_ids'].copy()
    return outputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)
print(f"   ✅ {len(tokenized_dataset)}件のデータをトークナイズしました")

# 学習の設定（調整版）
print("\n⚙️ 学習設定を構成中...")
training_args = TrainingArguments(
    output_dir="./iris_model",
    overwrite_output_dir=True,
    num_train_epochs=20,  # エポック数を増やす
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=1000,
    save_total_limit=2,
    learning_rate=3e-5,  # 学習率を下げる
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
    report_to="none",
    load_best_model_at_end=False,
)

# データコレーターの設定
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# トレーナーの初期化
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# 学習の実行
print("\n🚀 学習を開始します...")
print("   ⏱️ これには10-15分程度かかる場合があります...")
trainer.train()
print("   ✅ 学習完了！")

# モデルの保存
print("\n💾 モデルを保存中...")
trainer.save_model("./iris_model")
tokenizer.save_pretrained("./iris_model")

# config.jsonにモデルタイプを明示的に保存
import json
config_path = "./iris_model/config.json"
with open(config_path, 'r') as f:
    config = json.load(f)
config['model_type'] = 'gpt2'
config['task'] = 'iris_classification_chat'
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print("   ✅ ./iris_modelに保存しました")

In [ ]:
print("\n🧪 モデルのテスト...")
test_prompts = [
    "アヤメとは",
    "Setosaとは",
    "使い方を教えて",
    "典型的なVersicolorの値は？",
    "ありがとう"
]

model.eval()
for prompt in test_prompts:
    full_prompt = f"### 質問: {prompt}\n### 回答:"
    inputs = tokenizer.encode(full_prompt, return_tensors="pt")
    
    # GPUが使用可能な場合は使用
    if torch.cuda.is_available():
        inputs = inputs.cuda()
        model = model.cuda()
    
    with torch.no_grad():
        outputs = model.generate(
            inputs, 
            max_length=150,
            num_return_sequences=1,
            temperature=0.8,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_p=0.95,
            top_k=50,
            no_repeat_ngram_size=3
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response.replace(full_prompt, "").strip()
    
    # 最初の文だけを表示（長すぎる場合）
    if len(response) > 100:
        response = response[:100] + "..."
    
    print(f"\n   Q: {prompt}")
    print(f"   A: {response}")

In [ ]:
!zip -r iris_model.zip iris_model/
print("   ✅ iris_model.zipを作成しました")

print("\n📥 ダウンロードの準備...")
from google.colab import files
files.download('iris_model.zip')
print("   ✅ ダウンロードを開始します")

print("\n" + "=" * 60)
print("🎉 完了！")
print("iris_model.zipがダウンロードされました。")
print("このファイルを解凍して、Dockerコンテナの")
print("./llm/ディレクトリに配置してください。")
print("=" * 60)